In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [2]:
# 데이터 읽어오기
train_file = "./chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')

In [3]:
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


Q : Question   
A : Answer   
label   
0 - 일상다반사   
1 - 이별(부정)   
2 - 사랑(긍정)   

In [6]:
data.shape

(11823, 3)

In [7]:
features = data['Q'].tolist()
labels = data['label'].tolist()

In [10]:
features[:10]

['12시 땡!',
 '1지망 학교 떨어졌어',
 '3박4일 놀러가고 싶다',
 '3박4일 정도 놀러가고 싶다',
 'PPL 심하네',
 'SD카드 망가졌어',
 'SD카드 안돼',
 'SNS 맞팔 왜 안하지ㅠㅠ',
 'SNS 시간낭비인 거 아는데 매일 하는 중',
 'SNS 시간낭비인데 자꾸 보게됨']

In [12]:
labels[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [26]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]

In [28]:
corpus[:10] # 텍스트를 전처리 하여 말뭉치(corpus)로 만든 상태

[['12시', '땡'],
 ['1지망', '학교', '떨어졌어'],
 ['3박4일', '놀러가고', '싶다'],
 ['3박4일', '정도', '놀러가고', '싶다'],
 ['ppl', '심하네'],
 ['sd카드', '망가졌어'],
 ['sd카드', '안돼'],
 ['sns', '맞팔', '왜', '안하지ㅠㅠ'],
 ['sns', '시간낭비인', '거', '아는데', '매일', '하는', '중'],
 ['sns', '시간낭비인데', '자꾸', '보게됨']]

In [29]:
# tokenizer로 토큰화
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

In [31]:
sequences[:10]

[[4646, 4647],
 [4648, 343, 448],
 [2580, 803, 11],
 [2580, 804, 803, 11],
 [4649, 2581],
 [2582, 4650],
 [2582, 64],
 [805, 4651, 14, 4652],
 [805, 4653, 3, 502, 238, 45, 106],
 [805, 4654, 23, 4655]]

In [32]:
max(len(i) for i in sequences) # 단어 시퀀스의 최대 길이가 15임을 알아냄

15

In [33]:
# padding을 넣어 단어 시퀀스의 길이를 맞춤

MAX_SEQ_LEN = 15  # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [34]:
padded_seqs[:10]

array([[4646, 4647,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [4648,  343,  448,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [2580,  803,   11,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [2580,  804,  803,   11,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [4649, 2581,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [2582, 4650,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [2582,   64,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [ 805, 4651,   14, 4652,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [ 805, 4653,    3,  502,  238,   45,  106,    0,    0,    0,    0,
           0,    0,    0,    0],
       [ 805, 4654,   23, 4655,    0,

## 데이터셋 분할

In [35]:
# 학습용, 검증용, 테스트용 데이터셋 생성 ➌
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

## 하이퍼 파라미터 설정

In [36]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1  # 전체 단어 수

## CNN 모델 정의

In [37]:
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

In [38]:
conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)
conv3 = Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

In [39]:
# 3, 4, 5- gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(3, activation=tf.nn.softmax)(logits)

## 모델 생성

In [22]:
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## 모델 학습

In [40]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
414/414 [==============================] - 14s 34ms/step - loss: 0.0923 - accuracy: 0.9718 - val_loss: 0.0550 - val_accuracy: 0.9818
Epoch 2/5
414/414 [==============================] - 14s 35ms/step - loss: 0.0735 - accuracy: 0.9772 - val_loss: 0.0428 - val_accuracy: 0.9848
Epoch 3/5
414/414 [==============================] - 14s 34ms/step - loss: 0.0689 - accuracy: 0.9776 - val_loss: 0.0347 - val_accuracy: 0.9865
Epoch 4/5
414/414 [==============================] - 16s 40ms/step - loss: 0.0553 - accuracy: 0.9820 - val_loss: 0.0307 - val_accuracy: 0.9869
Epoch 5/5
414/414 [==============================] - 14s 35ms/step - loss: 0.0481 - accuracy: 0.9832 - val_loss: 0.0189 - val_accuracy: 0.9924


## 모델 평가

In [41]:
# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

60/60 [==============================] - 0s 4ms/step - loss: 0.0261 - accuracy: 0.9882
Accuracy: 98.815566
loss: 0.026111


In [42]:
# 모델 저장
model.save('cnn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
